In [ ]:
# LIBRARIES

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import scipy

from scipy import ndimage
from sklearn import preprocessing
import math


import tensorflow as tf
from tensorflow.python.framework import ops

In [ ]:
# DATASET

path_X_train = "./Data/X_train.csv"
X_train = pd.read_csv(path_X_train,header=0)
path_Y_train = "./Data/Y_train.csv"
Y_train = pd.read_csv(path_Y_train,header=0)

path_X_test = "./Data/X_test.csv"
X_test = pd.read_csv(path_X_test,header=0)
path_Y_test = "./Data/Y_test.csv"
Y_test = pd.read_csv(path_Y_test,header=0)

In [ ]:
X_test = pd.DataFrame.as_matrix(X_test)
X_train = pd.DataFrame.as_matrix(X_train)
Y_test = pd.DataFrame.as_matrix(Y_test)
Y_train = pd.DataFrame.as_matrix(Y_train)
Y_test = np.reshape(Y_test, [Y_test.shape[0],1])
Y_train = np.reshape(Y_train, [Y_train.shape[0], 1])

In [ ]:
def create_placeholders(n_x, n_y):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_x -- scalar, number of features
    n_y -- scalar, number of classes ({0,1}-->2)
    
    Returns:
    X -- placeholder for the data input, of shape [n_x, None] and dtype "float"
    Y -- placeholder for the input labels, of shape [n_y, None] and dtype "float" 
    """
    
    X = tf.placeholder(tf.float32, shape=(None, n_x))
    Y = tf.placeholder(tf.float32, shape=(None, n_y))
    
    return X, Y

In [ ]:
def initialize_parameters_lr(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l-1], layer_dims[l])
                    bl -- bias vector of shape (1,layer_dims[l])
    """
     
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = tf.get_variable('W' + str(l), [layer_dims[l-1], layer_dims[l]], initializer = tf.contrib.layers.xavier_initializer(), dtype=tf.float32)
        parameters['b' + str(l)] = tf.get_variable('b' + str(l), [1, layer_dims[l]], initializer = tf.zeros_initializer(), dtype=tf.float32)

        assert(parameters['W' + str(l)].shape == (layer_dims[l-1], layer_dims[l]))
        assert(parameters['b' + str(l)].shape == (1,layer_dims[l]))
    
    return parameters

In [ ]:
def forward_propagation_lr(X, parameters):
    """
    Implements the forward propagation for the model: LINEAR -> SOFTMAX
    
    Arguments:
    X -- input dataset placeholder, of shape (n_x, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1"
                  the shapes are given in initialize_parameters

    Returns:
    Z1 -- the output of the LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    b1 = parameters['b1']
    
            
    Z1 = tf.add(tf.matmul(X, W1),b1)
    
    
    return Z1

In [ ]:
def compute_cost_lr(Z1, Y):
    """
    Computes the cost
    
    Arguments:
    Z1 -- output of forward propagation (output of the LINEAR unit), of shape (n_y, number of examples)
    Y -- "true" labels vector placeholder, same shape as Z1
    
    Returns:
    cost - Tensor of the cost function
    """
   
    logits = Z1
    labels = Y
    
  
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = logits, labels = labels))

    
    return cost

In [ ]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:]
    shuffled_Y = Y[permutation,:].reshape((m, Y.shape[1]))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size, :]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size, :]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m, :]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m, :]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [ ]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.001,
          num_epochs = 100, minibatch_size = 32, print_cost = True):
    """
    Implements a one-layer tensorflow neural network: LINEAR->SIGMOID.
    
    Arguments:
    X_train -- training set, of shape (input size = n_x, number of training examples = train_m)
    Y_train -- test set, of shape (output size = n_y, number of training examples = train_m)
    X_test -- training set, of shape (input size = n_x, number of training examples = test_m)
    Y_test -- test set, of shape (output size = n_y, number of test examples = test_m)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    seed = 3                                          # to keep consistent results
    (m, n_x) = X_train.shape                          # (n_x: input size, m : number of examples in the train set)
    n_y = Y_train.shape[1]                            # n_y : output size
    costs = []                                        # To keep track of the cost
    
    # Create Placeholders of shape (n_x, n_y)
    X, Y = create_placeholders(n_x, n_y)
   

    # Initialize parameters
    layer_dims = [n_x, n_y]
    parameters = initialize_parameters_lr(layer_dims)
    
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    Z1 = forward_propagation_lr(X, parameters)
    
    # Cost function: Add cost function to tensorflow graph
    cost = compute_cost_lr(Z1, Y)
    
    # Backpropagation
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    gvs = optimizer.compute_gradients(cost)
    train_op = optimizer.apply_gradients(gvs)
   


    
    # Initialize all the variables
    init = tf.global_variables_initializer()

    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        
        # Do the training loop
        for epoch in range(num_epochs):
            epoch_cost = 0.                       # Defines a cost related to an epoch
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
            count=0
            
            for minibatch in minibatches:
                count+=1
                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                             
                # Run the session to execute the "optimizer" and the "cost", the feedict should contain a minibatch for (X,Y).
                _ , minibatch_cost = sess.run([train_op, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
   
                epoch_cost += minibatch_cost / num_minibatches

            # Print the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
                
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # lets save the parameters in a variable
        parameters_lr = sess.run(parameters)
        print ("Parameters have been trained!")

        # Calculate the correct predictions
        raw_prediction_lr = tf.sigmoid(Z1)
        prediction = raw_prediction_lr >0.5
        correct_prediction = tf.equal(tf.cast(prediction, tf.int32), tf.cast(Y, tf.int32))
         
        
        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        

        print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
  
        return parameters_lr, raw_prediction_lr.eval({X: X_test, Y: Y_test})

In [ ]:
parameters_lr, test_predictions_lr = model(X_train[:,1:] , Y_train, X_test[:,1:], Y_test)

In [ ]:
# SAVE PREDICTIONS

np.savetxt('test_lr', test_predictions_lr, delimiter=',')  

In [ ]:
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp



# Binarize the output
y = label_binarize(Y_test, classes=[0, 1])
y_score = test_predictions_lr
n_classes = y.shape[1]





# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])




In [ ]:
# AUC

roc_auc[0]

In [ ]:
# CONFUSION MATRIX

ypreds = test_predictions_lr
ypreds[ypreds<=0.5]=0
ypreds[ypreds>0.5]=1

from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, ypreds)
tn, fp, fn, tp = confusion_matrix(Y_test, ypreds).ravel()

In [ ]:
# PRECISION

tp/(tp+fp)

In [ ]:
# RECALL

tp/(tp+fn)

In [ ]:
# ACCURACY

(tp+tn)/(6000)

In [ ]:
# ROC, AUC, GINI

plt.figure()
lw = 1
plt.plot(fpr[0], tpr[0], color='darkorange',
         lw=lw, label='ROC curve (area = %0.3f)' % roc_auc[0])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()


In [ ]:
# OCCLUSION TEST

In [ ]:
# TP AND TN EXAMPLES

X_test[5736, :] # X_test[5736, :]--> TP, X_test[4669, :] -->  TN

In [ ]:
# MEAN VALUES TO REPLACE EACH FEATURE IN TURN WITH

X = np.concatenate([X_train, X_test], axis=0)
xm = np.mean(X, axis = 0)
xm = np.reshape(xm, [1, 24])
xm

In [ ]:
# MODIFIED SIGMOID SCORE FOR EACH FEATURE

z=[]
count=0

for i in range(1, 24):
    x = list(X_test[4669, :])
    X = np.reshape(x, [1,24])
    a = xm[0, i]
    #print("count", count)
    #print("a", a)
    #print("x", x)
    X[0, i] = a
    
    X = tf.cast(X[:, 1:], tf.float32)    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters_lr['W1']
    b1 = parameters_lr['b1']
    
    Z1 = tf.add(tf.matmul(X, W1),b1)                                          
    A1 = tf.sigmoid(Z1)
    z.append(A1)
    count=count+1
    
    #z[:, i] = output_grad


with tf.Session() as sess:
    score = sess.run(z)
    flat_list = [item for sublist in score for item in sublist]
    flat_list = [item for sublist in flat_list for item in sublist]
    print(flat_list)

In [ ]:
# PLOT THE TP AND TN EXAMPLES ON THE HEATMAP

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.DataFrame({"TN [0.227]":flat_list},
                  index=["LIMIT_BAL", "GENDER", "EDUCATION", "MARITAL_STATUS", "AGE", "PAY_1", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6", 
"BILL_AMT1", "BILL_AMT2","BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", 
"PAY_AMT5", "PAY_AMT6"])
plt.subplots(figsize=(20,2))
sns.heatmap(df.T, annot=True,annot_kws={"size":10}, fmt=".3f", cmap='Blues', linewidth=2, linecolor='white', cbar=True)
